In [269]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

pd.options.mode.chained_assignment = None  # avoid slide-copy-warning 

In [270]:
df = pd.read_csv("titanic.csv")
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [271]:
df_selection = df[["Survived", "Pclass", "Age", "SibSp", "Parch", "Fare", "Embarked"]]
df_selection.head()

,Survived,Pclass,Age,SibSp,Parch,Fare,Embarked
0,0,3,22.0,1,0,7.2500,S
1,1,1,38.0,1,0,71.2833,C
2,1,3,26.0,0,0,7.9250,S
3,1,1,35.0,1,0,53.1000,S
4,0,3,35.0,0,0,8.0500,S


In [272]:
one_hot_feature_1 = pd.get_dummies(df_selection["Embarked"])
one_hot_feature_1.head()
df_selection = df_selection.drop(columns=["Embarked"])
df_selection = pd.concat([df_selection, one_hot_feature_1], axis=1)
df_selection.head()

,Survived,Pclass,Age,SibSp,Parch,Fare,C,Q,S
0,0,3,22.0,1,0,7.2500,0,0,1
1,1,1,38.0,1,0,71.2833,1,0,0
2,1,3,26.0,0,0,7.9250,0,0,1
3,1,1,35.0,1,0,53.1000,0,0,1
4,0,3,35.0,0,0,8.0500,0,0,1


In [273]:
df_X = df_selection.drop(columns = ["Survived"])
df_y = df_selection["Survived"]
X_train, X_test, y_train, y_test = train_test_split(df_X, df_y, test_size=0.2, random_state=0)

### Augabe 1: Fehlende Werte auffüllen
- Überprüfen Sie welche Attribute in `X_train` N/A-Werte enthalten
- Füllen Sie diese Werte mit dem Durchschnittswerte der übrigen Werte aus der gleichen Spalte auf

### Aufgabe 2: Daten skalieren
- Skalieren Sie die Daten aus `X_train` mit einem `StandardScaler`

### Aufgabe 3:  Modelltraining
- Trainieren Sie das Modell auf dem skalierten `X_train` und `y_test`

### Aufgabe 4: Evaluation
- Entfernen Sie alle Datenpunkte mit N/A-Werten aus `X_test`
- Entfernen Sie die genau gleichen Zeilen auch aus `y_test`
- Skalieren Sie die Datenpunkte `X_test` mit dem gleichen skalieren wie in Aufgabe 2
- Machen Sie eine Vorhersage auf den skalierten Testdaten und berechnen sie Accuracy, Precision und Recall

### Aufgabe 5. Neues Feature `Embarked`
- Fügen Sie `Embarked` als weiteres Attribut zu `df_selection` hinzu (Zelle 3). 
- Führen Sie ein one-hot-encoding dieses Attributs durch und durchlaufen Sie Schritte aus Aufgabe 1 bis 4.
- Vergleichen Sie die Performance des Modells mit und ohne dieses Feature.

In [274]:
X_train.isna().sum()

Pclass      0
Age       141
SibSp       0
Parch       0
Fare        0
C           0
Q           0
S           0
dtype: int64

In [275]:
mean_age = X_train['Age'].mean()
print(mean_age)
X_train['Age'] = X_train['Age'].fillna(mean_age)
X_train.isna().sum()

29.745183887915935


Pclass    0
Age       0
SibSp     0
Parch     0
Fare      0
C         0
Q         0
S         0
dtype: int64

scaler

In [276]:
scaler = StandardScaler().fit(X_train)
X_scaled = scaler.transform(X_train)
X_scaled

array([[ 0.81925059,  0.        , -0.46445234, ...,  2.12588331,
        -0.31426968, -1.62827579],
       [-0.38096838,  0.09593203, -0.46445234, ..., -0.4703927 ,
        -0.31426968,  0.61414657],
       [-0.38096838,  0.09593203,  0.41270964, ...,  2.12588331,
        -0.31426968, -1.62827579],
       ...,
       [ 0.81925059,  0.        , -0.46445234, ..., -0.4703927 ,
         3.18198052, -1.62827579],
       [ 0.81925059,  0.47818735,  0.41270964, ..., -0.4703927 ,
        -0.31426968,  0.61414657],
       [-0.38096838,  2.31301288,  0.41270964, ..., -0.4703927 ,
        -0.31426968,  0.61414657]])

In [277]:
from sklearn.linear_model import LogisticRegression

logisticRegr = LogisticRegression(max_iter=1000)
logisticRegr.fit(X_scaled, y_train)

LogisticRegression(max_iter=1000)

drop from x test

In [278]:
X_test = X_test.dropna()
X_test.shape

(143, 8)

drop from y test

In [279]:
y_test = y_test[X_test.index]
y_test.shape

(143,)

scale tests

In [280]:
X_test_scaled = scaler.transform(X_test)
#X_test_scaled

In [281]:
from sklearn.metrics import mean_squared_error

predictions = logisticRegr.predict(X_test_scaled)
mse =  mean_squared_error(y_test, predictions)
print("Mean squared error (MSE) auf Testdaten: %.2f" % mse)

Mean squared error (MSE) auf Testdaten: 0.25


In [282]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, predictions)

0.7482517482517482

In [283]:
from sklearn.metrics import precision_score
precision_score(y_test, predictions)

0.7948717948717948

In [284]:
from sklearn.metrics import recall_score
recall_score(y_test, predictions)

0.5254237288135594